这里记录Speck开发过程中遇到的问题和解决方案。

产品具体型号：Speck 2E DevKit，
开发环境：Python3.92 + Macbook macOS 11 + CPU + USB 3.0
Speck不支持Windows系统，具体是指Speck的Samna库不支持WIndows。所以如果使用Windows可以进行仿真模拟，但是无法实际配置到Speck开发板上。（时识科技的官方文档中没有说明过是否支持Windows，但是根据我个人的尝试，Samna无法在Windows上运行。如果后续有人运行成功，请联系我更新文档）
Speck支持macOS和Linux系统，macOS系统上会出现一些Linux系统上没有的特定问题，这些我也会在后续说明。


Speck最基础的使用逻辑是通过ANN训练一个神经网络，然后使用Speck自带的库sinabs-dynapcnn将保存好的模型参数加载到SNN上，最后使用Speck自带的库samna将SNN转换为硬件可接受的SNN模型并部署到开发板上。

于是，我们就可以发现对于Speck来说需要的只是一个模型参数文件（当然这个参数文件需要被仔细计算过，需要保证和Speck硬件一一对应，否则无法部署，这个后续再讲）。那么训练这个模型文件的方法就有很多种。个人使用过的有三种训练方式：

1. 直接训练ANN：这是官方推荐的最常用的方式，个人验证后也是效果最好的一种方式。但是使用这种方式有时会导致输出脉冲序列为空。这可能是spike拥塞导致的。这一问题个人没有解决方案，但是ANN这一方法上全局来看这一问题的影响不大。

2. STDP训练SNN：个人不推荐使用局部学习算法，因为可能会导致spike逐层稀疏，从而也会导致输出层的脉冲序列为空。

3. STBP训练SNN：个人最不推荐的学习方式，训练速度很慢，而且效果没有ANN2SNN好。如果研究需要可以选择这一方式。但是个人训练的STBP模型上经常会有输出层的脉冲序列为空的问题。

Problem: 

Windows自带Linux系统WSL Linux底层交互的samna toolkit在新环境中无法直接检测到板子，需要手动修改一下Linux内核的设备管理器，向udev里添加一个vid，但是这一步官方文档给的指导有点问题。

解决方案：使用CentOS配置成功，建议尽量在纯Linux环境中或者MacOS环境搭建。

Problem：

samnagui已经集成到samna库中，可以直接import，也可以通过内置命令行引用。

其中，macOS不能按照官网文档进行调用，也无法进行直接调用。唯一调用方式：在代码中使用命令行。

Problem：

使用samnagui 的 visualizer进行DVS相机交互时，请在每次启动前手动清空所有设置的节点buffer，或者冷重启（不建议）。否则会导致：相机正常运行，能耗显示为空，产生脉冲为空，程序不报错。

Problem：

部署神经网络到开发版时不要直接使用 .to(device) 方法，需要手动配置samna源溯节点和configuration(量化权重和阈值)，要保证每次运行时write buffer和read buffer之间有 >=3.0s 的时间差，否则会导致：节点无法读入事件流，预测输出为空，程序不报错。后续发现这是由于Speck上spike拥塞。spike拥塞也会出现其他症状，会在后续一一提及。

Note:

在MNIST task中，层数为3～4层时表现最好。Speck内部有9个可配置的卷积层，但是实际使用时不宜超过5层。因为随着层数的增加，理论上中间层会越来越宽，leading to increasing number of parameters。Speck的9个core中，最大的核kernel memory为64 Ki (core 5, core 6)，在配置large parameter layer时内存受限。SNN的片下测试中有一套表现极好的参数 (acc = 98%)，但是囿于内存限制无法配置到板子上。

Note：

猜测实际kernel memory可能略大于标注值。理论计算65Ki需求量的参数可以配置到64Ki容量的卷积层上。

Note：

注意DVS相机架构问题，IBM数据集使用的是随机输出事件，时间戳对应当时变化的时间。如果使用按行输出事件的DVS，会造成事件输出顺序和实际触发顺序不一致，降低模型分类准确率。

Problem：

使用STBP直接训练SNN，并且转换为Sequential格式搭载到板子上的方案可行，但是会遇到开发板输出脉冲全0的情况。这也是由于spike拥塞导致的。

Problem：

调用Speck内置DVS相机时又是会出现画面全横条的问题，是由于spike拥塞。

关于Speck的spike拥塞问题：

简单来讲就是脉冲信号密度太大，导致芯片堵了。

Speck官方没有预制的filter，来过滤部分脉冲信号，即16过1，128过1等等。使用内置的DVS相机进行拍摄的时候，最好在preprocessing阶段就设置这样一个filter，因为如果使用SNN的第一层去做过滤的话个人感觉不是很有效果。

但是这个方法是不可行的，因为Synsense的工程师讲预制的预处理层是没有这一功能的。如果需要这样的filter，需要将DVS信号全部导入外部处理器进行处理，然后再进行传输。不过不用觉得太麻烦，因为如果想把这个片子的精度飙高，像这样频繁的读出处理再部署是必不可少的。

总而言之，如果出现spike拥塞问题，我摸索出三个解决方案：

1. DVS的事件（或者其他数据集）在计算机中过滤好再发送回Speck
2. 降低SNN权重，减少脉冲事件。这个实际操作比较难。
3. SNN的后面再过一个filter，减少事件密度。

以上便是我使用Speck阶段遇到的所有问题了。其实出现这些问题的主要原因还是个人能力不足，遇到的时候没有经验解决。因为这些都是在任何一个开发板上可能会出现的很general的问题。整体来说问题不是很多。

我个人在使用Speck过程中遇到的最大的问题是官方的API文档不全，很多已知的功能不知道怎样使用，很多已知的API也不知道其调用的方式，包括samna的部署逻辑都是从几段代码片段里摸索出来的。所以那个时候亟需的是一个教程或者完整的document。这也是我写这个项目的最初目的，即教程。然而这几天再度回到官方文档中，发现这半年文档一直在被更新，现在已经非常全面了。估计用不了多久所有提到的问题也都会在官方文档中被一一解决。